In [48]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from itertools import product
!pip install kmodes

In [66]:
#we will first check for all columns which have null valuues
missing_values = ['    ', 'NA', 'N/A', 'NaN', 'nan', 'NULL', 'null', 'None', 'none']
variables=pd.read_csv('DATA/Dades_ministeri_transport.csv')
# Replace whitespace characters with NaN in the "discharge_date" column
df=variables
# List of columns where you want to replace whitespace characters with NaN
columns_to_clean = df.columns.tolist()
  # Add all relevant column names here

# Replace whitespace characters with NaN in the specified columns
#for col in columns_to_clean:
    #variables[col] = variables[col].replace(r'^\s*$', np.nan, regex=True)
def clean_whitespace(value):
    if isinstance(value, str):
        return value.strip() if value.strip() else np.nan
    return value

# Apply the cleaning function to every element in the DataFrame with applymap
df = df.applymap(clean_whitespace)
    
#variables['discharge_date'] = variables['discharge_date'].replace(r'^\s*$', np.nan, regex=True)
df.info()
#variables =variables[variables["contract_type"] == 401]
#variables["diagnoses_code"].str.strip()
#variables[variables["diagnoses_code"].iseq("")]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411103 entries, 0 to 411102
Data columns (total 58 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   accident_type                          411103 non-null  int64  
 1   gender                                 411103 non-null  int64  
 2   nationality                            411103 non-null  int64  
 3   work_situation                         411103 non-null  int64  
 4   main_occupation                        411103 non-null  int64  
 5   seniority_in_the_job_months            411103 non-null  int64  
 6   seniority_in_the_job_days              411103 non-null  int64  
 7   contract_type                          411103 non-null  int64  
 8   social_security_regime                 411103 non-null  int64  
 9   economic_activity_center               411103 non-null  int64  
 10  number_workers_center                  411103 non-null  

In [67]:
#Now we are going to create columns with the dates in the correct format
def convert_to_datetime(row):
    converted_dates = []
    for col in ['date_accident', 'date_medical_leave', 'discharge_date']:
        int_date_str = str(row[col])
        day = int_date_str[:-6]
        month = int_date_str[-6:-4]
        year = int_date_str[-4:]
        try:
            converted_date = pd.to_datetime(f"{day}/{month}/{year}", format='%d/%m/%Y')
        except ValueError:
            converted_date = pd.NaT
        converted_dates.append(converted_date)
    return pd.Series(converted_dates, index=['DateColumn_accident', 'DateColumn_medical_leave', 'DateColumn_discharge'])

# Apply the function to each row
df[['DateColumn_accident', 'DateColumn_medical_leave', 'DateColumn_discharge']] = df.apply(convert_to_datetime, axis=1)
df

,accident_type,gender,nationality,work_situation,main_occupation,seniority_in_the_job_months,seniority_in_the_job_days,contract_type,social_security_regime,economic_activity_center,...,daily_temporary_disability_subsidy,injury_severity_discharged,discharge_date,diagnoses_code,leave_duration,age,year_accident,DateColumn_accident,DateColumn_medical_leave,DateColumn_discharge
0,1,1,724,1,843,36,0,100,1,494,...,36.98,1,4122017,9590,63,32,2017,2017-10-03,2017-10-03,2017-12-04
1,1,1,170,1,841,0,8,401,1,494,...,43.81,1,17092017,85221,40,43,2017,2017-08-09,2017-08-09,2017-09-17
2,1,1,724,1,843,4,0,189,1,494,...,31.61,1,2052017,84502,14,32,2017,2017-04-19,2017-04-19,2017-05-02
3,1,1,724,1,841,14,0,289,1,471,...,20.09,1,2092017,7242,3,32,2017,2017-08-30,2017-08-31,2017-09-02
4,1,1,724,1,843,25,0,502,1,494,...,18.54,1,5012018,84200,65,32,2017,2017-11-02,2017-11-02,2018-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411098,1,1,100,1,843,149,0,189,1,494,...,36.25,2,27092020,80709,111,58,2020,2020-06-09,2020-06-09,2020-09-27
411099,1,1,604,1,844,8,0,200,1,561,...,9.45,1,4102020,9221,3,50,2020,2020-09-30,2020-10-02,2020-10-04
411100,1,1,642,1,843,160,0,100,1,172,...,40.13,1,16082020,81501,115,38,2020,2020-04-24,2020-04-24,2020-08-16
411101,1,1,642,1,832,48,0,100,1,16,...,40.82,1,10072020,9230,23,32,2020,2020-06-18,2020-06-18,2020-07-10


In [59]:
#df[['date_accident', 'date_medical_leave', 'discharge_date','DateColumn_accident', 'DateColumn_medical_leave', 'DateColumn_discharge']]
variables=df[['gender','work_situation','main_occupation','seniority_in_the_job_months','age','province','workers','economic_activity_type','DateColumn_accident','week_day','time_workday','place_type','job_type','injury_severity']]
variables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411103 entries, 0 to 411102
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   gender                       411103 non-null  int64         
 1   work_situation               411103 non-null  int64         
 2   main_occupation              411103 non-null  int64         
 3   seniority_in_the_job_months  411103 non-null  int64         
 4   age                          411103 non-null  int64         
 5   province                     411103 non-null  int64         
 6   workers                      411103 non-null  int64         
 7   economic_activity_type       411103 non-null  int64         
 8   DateColumn_accident          411103 non-null  datetime64[ns]
 9   week_day                     411103 non-null  int64         
 10  time_workday                 411103 non-null  int64         
 11  place_type                

In [68]:
variables = df[['gender', 'work_situation', 'main_occupation', 'seniority_in_the_job_months', 'age', 'province', 'workers', 'economic_activity_type', 'date_accident', 'week_day', 'time_workday', 'place_type', 'job_type', 'injury_severity']]

# Mapping for injury severity
injury_mapping = {
    1: 'mild',
    2: 'serious',
    3: 'very serious',
    4: 'mortal',
    '404': 'severity not known'
}
variables['injury_severity'] = variables['injury_severity'].map(injury_mapping).fillna('severity not known')

# Categorize age using labels directly
age_categories = ['16-25', '26-35', '36-45', '46-55', '+56', 'Age not known']
variables['age'] = pd.Categorical(variables['age'], categories=age_categories, ordered=True)

# Unique values in 'main_occupation' column
x = variables['main_occupation'].unique()
print(x)

# Frequency and percentage of 'main_occupation' values
value_counts = variables['main_occupation'].value_counts()
percentage = (value_counts / len(variables)) * 100
print(percentage)

[843 841 834 831 833 842 844 832]
843    47.237067
841    21.484640
833    11.261655
844     7.981455
842     6.407397
832     2.660161
831     1.635113
834     1.332513
Name: main_occupation, dtype: float64


<ipython-input-68-02fc5e2d03c8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variables['injury_severity'] = variables['injury_severity'].map(injury_mapping).fillna('severity not known')
<ipython-input-68-02fc5e2d03c8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variables['age'] = pd.Categorical(variables['age'], categories=age_categories, ordered=True)


In [69]:
df['Quarter'] = df['DateColumn_accident'].apply(lambda x: pd.Timestamp(x).quarter)
df

,accident_type,gender,nationality,work_situation,main_occupation,seniority_in_the_job_months,seniority_in_the_job_days,contract_type,social_security_regime,economic_activity_center,...,injury_severity_discharged,discharge_date,diagnoses_code,leave_duration,age,year_accident,DateColumn_accident,DateColumn_medical_leave,DateColumn_discharge,Quarter
0,1,1,724,1,843,36,0,100,1,494,...,1,4122017,9590,63,32,2017,2017-10-03,2017-10-03,2017-12-04,4
1,1,1,170,1,841,0,8,401,1,494,...,1,17092017,85221,40,43,2017,2017-08-09,2017-08-09,2017-09-17,3
2,1,1,724,1,843,4,0,189,1,494,...,1,2052017,84502,14,32,2017,2017-04-19,2017-04-19,2017-05-02,2
3,1,1,724,1,841,14,0,289,1,471,...,1,2092017,7242,3,32,2017,2017-08-30,2017-08-31,2017-09-02,3
4,1,1,724,1,843,25,0,502,1,494,...,1,5012018,84200,65,32,2017,2017-11-02,2017-11-02,2018-01-05,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411098,1,1,100,1,843,149,0,189,1,494,...,2,27092020,80709,111,58,2020,2020-06-09,2020-06-09,2020-09-27,2
411099,1,1,604,1,844,8,0,200,1,561,...,1,4102020,9221,3,50,2020,2020-09-30,2020-10-02,2020-10-04,3
411100,1,1,642,1,843,160,0,100,1,172,...,1,16082020,81501,115,38,2020,2020-04-24,2020-04-24,2020-08-16,2
411101,1,1,642,1,832,48,0,100,1,16,...,1,10072020,9230,23,32,2020,2020-06-18,2020-06-18,2020-07-10,2


In [72]:
#df.to_csv('DATA\Transport_data_clean.csv', index=False) 

In [70]:

categorical = ['gender','main_occupation','seniority_in_the_job_months','age','province','workers','economic_activity_type','date_accident','week_day','time_workday','place_type','job_type','injury_severity','DateColumn_accident']
import pandas as pd
import itertools 

#Create a list of diferent combinations of 3 variables of the given variables in categorical
combinations = list(itertools.combinations(categorical, 3))

# Load the dataset into a Pandas DataFrame

#Create a dictonary that has the results for each combination of variables

dic={}

# Count the number of occurrences of each unique combination of values
for combination in combinations:
    counts = df.groupby(list(combination)).size().reset_index(name='count')
    counts['percentage %'] = counts['count'] / df.shape[0] * 100

    # Get the top 10 combinations of values that are the most repeated in the dataset
    top_counts = counts.sort_values('count', ascending=False).head(10)
    dic[tuple(combination)]=counts.sort_values('count', ascending=False)
dic3=dic
# It gives the combinations of values that are freqüent in more that 10 percent of all the accidents.
for key, value in dic.items():
    condition = value.iloc[:, -1] > 10
    result = value[condition]
    
    if not result.empty:
        print(result)
        print('-----------------------------------------------------')

      gender  main_occupation  economic_activity_type   count  percentage %
1075       1              843                     494  100593      24.46905
-----------------------------------------------------
     gender  main_occupation  place_type  count  percentage %
272       1              843          13  76762     18.672206
-----------------------------------------------------
     gender  main_occupation  job_type  count  percentage %
195       1              843        12  62956     15.313924
224       1              843        61  49257     11.981669
-----------------------------------------------------
    gender  main_occupation  injury_severity   count  percentage %
24       1              843                1  188400     45.827931
16       1              841                1   81149     19.739335
8        1              833                1   43527     10.587858
-----------------------------------------------------
     gender  province  injury_severity  count  percentage %


In [87]:
import pandas as pd
import itertools 

categorical = ['gender','main_occupation','seniority_in_the_job_months','age','province','workers','economic_activity_type','date_accident','week_day','time_workday','place_type','job_type','injury_severity','date']

#Create a list of diferent combinations of 4 variables of the given variables in categorical
combinations = list(itertools.combinations(categorical, 4))

# Load the dataset into a Pandas DataFrame
df = variables

#Create a dictonary of the top 10 counts for each combination

dic={}

# Count the number of occurrences of each unique combination of values
for combination in combinations:
    counts = df.groupby(list(combination)).size().reset_index(name='count')
    counts['percentage %'] = counts['count'] / df.shape[0] * 100

    # Get the top 10 combinations of values that are the most repeated in the dataset
    top_counts = counts.sort_values('count', ascending=False).head(10)
    dic[tuple(combination)]=counts.sort_values('count', ascending=False)
dic4=dic 
# It gives the combinations of 4 values that are freqüent in more that 5 percent of all the accidents.
for key, value in dic.items():
    condition = value.iloc[:, -1] > 5
    result = value[condition]
    
    if not result.empty:
        print(result)
        print('-----------------------------------------------------')

      gender  main_occupation    age  economic_activity_type  count  \
2336       1              843  36-45                     494   8239   
2510       1              843  46-55                     494   7943   

      percentage %  
2336      8.832074  
2510      8.514767  
-----------------------------------------------------
     gender  main_occupation    age  place_type  count  percentage %
902       1              843  36-45          13   6393      6.853192
949       1              843  46-55          13   6017      6.450126
-----------------------------------------------------
     gender  main_occupation    age  job_type  count  percentage %
754       1              843  36-45        12   5438      5.829447
785       1              843  46-55        12   4764      5.106930
-----------------------------------------------------
     gender  main_occupation    age injury_severity  count  percentage %
102       1              843  36-45            mild  15067     16.151578
106    

In [88]:
import pandas as pd
import itertools 

#Create a list of diferent combinations of 6 variables of the given variables in categorical
combinations = list(itertools.combinations(categorical, 6))

# Load the dataset into a Pandas DataFrame
df = variables

#Create a dictonary of the top 10 counts for each combination

dic={}

# Count the number of occurrences of each unique combination of values
for combination in combinations:
    counts = df.groupby(list(combination)).size().reset_index(name='count')
    counts['percentage %'] = counts['count'] / df.shape[0] * 100

    # Get the top 10 combinations of values that are the most repeated in the dataset
    top_counts = counts.sort_values('count', ascending=False).head(10)
    dic[tuple(combination)]=counts.sort_values('count', ascending=False)
dic6=dic   
# It gives the combinations of 6 values that are freqüent in more that 1 percent of all the accidents.
for key, value in dic.items():
    condition = value.iloc[:, -1] > 1
    result = value[condition]
    
    if not result.empty:
        print(result)
        print('-----------------------------------------------------')

      gender  main_occupation    age  economic_activity_type  week_day  \
7907       1              843  36-45                     494         1   
8669       1              843  46-55                     494         1   
7911       1              843  36-45                     494         2   
8672       1              843  46-55                     494         2   
7915       1              843  36-45                     494         3   
8676       1              843  46-55                     494         3   
7919       1              843  36-45                     494         4   
8684       1              843  46-55                     494         5   
7923       1              843  36-45                     494         5   
8680       1              843  46-55                     494         4   

     injury_severity  count  percentage %  
7907            mild   1820      1.951010  
8669            mild   1688      1.809508  
7911            mild   1521      1.630487  
8672     